In [140]:
from tkinter import Canvas, Label, Tk, Frame

In [141]:
# reload module
%load_ext autoreload
%autoreload 2

from calculations import Calculations
from point import Point
from cursor_manager import CursorManager

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [142]:
root = Tk()

canvas = CurveGrid(root, width=600, height=650, bg="#333333")
canvas.pack(expand=True, fill="both")
canvas.configure(bg="#333333")

root.title("сurves")
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Python310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\grayni\AppData\Local\Temp\ipykernel_16264\1526301546.py", line 80, in on_enter
    self.delete_cursor(event)
TypeError: CursorManager.delete_cursor() takes 1 positional argument but 2 were given
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Python310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\grayni\AppData\Local\Temp\ipykernel_16264\1526301546.py", line 80, in on_enter
    self.delete_cursor(event)
TypeError: CursorManager.delete_cursor() takes 1 positional argument but 2 were given
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Python310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\grayni\AppData\Local\Temp\ipykernel_16264\1526301546.py", line 80, in 

In [143]:
class CurveGrid(Canvas, Frame, Calculations, Point, CursorManager):

    def __init__(self, master=None, **kwargs):
        super().__init__(master, **kwargs)
        CursorManager.__init__(self)
        self.root = master
        # hover
        self.bind("<Motion>", self.on_enter)
        # (optional: keyboard + ) LBM
        self.bind("<Button-1>", self.left_click)
        # click and hold
        self.bind("<B1-Motion>", self.on_drag)
        # left mouse button release
        self.bind("<ButtonRelease-1>", self.on_release)
        self.ctrl_pressed = False
        self.root.bind("<KeyPress>", self.ctrl_ready)
        self.bind('<Control-KeyRelease>', self.arrow_cursor)


        self.selected_index = None
        self.lim = {
            'x': 8,
            'y': 63,
            'dist_remove': 6
        }
        self.x = self.y = 0
        self.start_x = self.end_y = 50
        self.end_x = self.start_y = 560
        self.start_point = (self.start_x, self.start_y)
        self.end_point = (self.end_x, self.end_y)

        self.points = [self.start_point, self.start_point, (150, 200), (250, 100), self.end_point, self.end_point]
        self.curve_points = None

        # create Label coords,
        self.label = Label( master, text=f"x: 0\ny: 0", anchor="w", justify="left", bg="#333333", fg="#aaaaaa", font=("Arial", 14))
        self.label.place(x=50, y=580)

        # init grid & line
        self.draw_grid()
        self.draw_line()

    def on_key_release(self, event):
        if event.keysym == 'Control_L' or event.keysym == 'Control_R':
            self.ctrl_pressed = False
            print("Клавиша Ctrl отпущена")

    def draw_grid(self):
        # vertical lines
        for x in range(50, 561, 102):
            self.create_line(x, 50, x, 561, fill="#444444")

        # horizontal lines,
        for y in range(50, 561, 102):
            self.create_line(50, y, 561, y, fill="#444444")

    def draw_line(self):
        self.delete("curve", "point")

        # curve coords
        self.curve_points = self.curve_coords(self.points[1:-1])

        # curve
        self.create_line(
            # left fix point, curve points, right fix point
            self.points[0], *self.curve_points, self.points[-1],
            fill="#ffffbb",
            width=1,
            tags="curve"
        )

        # points
        for i, (x, y) in enumerate(self.points, start=0):
            if i not in (0, len(self.points)-1):
                self.draw_point(x, y) # class Point

    def on_enter(self, event):
        # hover points
        if event.state & 0x4:
            self.delete_cursor(event)
        if self.allow_add_point(event):
            self.hand_cursor(event) if self.allow_influence_point(event) else self.cross_cursor(event)
        else:
            if self.allow_influence_point(event):
                if event.state & 0x4:
                    self.delete_cursor(event)
                else:
                    self.hand_cursor(event)
            else:
                self.arrow_cursor(event)

    def left_click(self, event):
        self.x, self.y = event.x, event.y

        # ctrl + LMB
        if event.state & 0x4:
            if self.allow_influence_point(event):
                self.remove_point()
                self.draw_line()
        else:
            # add a point to the curve & redraw the curve
            if self.allow_add_point(event):
                self.add_point(self.x, self.y)
                self.draw_line()

            # choose the nearest point
            self.choose_point(self.x, self.y)

    # drag point,
    def on_drag(self, event):
        self.configure(cursor=self.drag_path)
        # event.widget.configure(cursor="fleur")
        # restricted movement coordinates
        self.x = max(self.start_x, min(event.x, self.end_x))
        self.y = max(self.end_y, min(event.y, self.start_y))

        # selected point
        self.points[self.selected_index] = (self.x, self.y)

        # coord: x -> next point, previous point
        prev_x, _ = self.points[self.selected_index - 1]
        next_x, _ = self.points[self.selected_index + 1]

        # limit min distance between points
        if self.selected_index != 1 and self.x - prev_x < self.lim['x']:
            self.limited_point(prev_x + self.lim['x'])
        elif self.selected_index != len(self.points)-2 and next_x - self.x < self.lim['x']:
            self.limited_point(next_x - self.lim['x'])

        self.label.config(text=f"x: {round((self.x - 50) / 2)}\ny: {round((560 - self.y) / 2)}")

        # boundary invisible points
        if self.selected_index == 1:
            self.points[0] = (self.start_x, self.y)
        elif self.selected_index == len(self.points) - 2:
            self.points[-1] = (self.end_x, self.y)

        self.draw_line()

    def on_release(self, event):
        self.selected_index = None
        self.arrow_cursor(event)

In [144]:
# points = [(50, 560), (150, 200), (250, 100), (560, 50)]

In [145]:
import tkinter as tk

class MyApp:
    def __init__(self, root):
        self.root = root
        self.ctrl_pressed = False
        self.root.bind("<KeyPress>", self.on_key_press)
        self.root.bind("<KeyRelease>", self.on_key_release)

    def on_key_press(self, event):
        if event.keysym == 'Control_L' or event.keysym == 'Control_R':
            self.ctrl_pressed = True
            print("Клавиша Ctrl нажата")

    def on_key_release(self, event):
        if event.keysym == 'Control_L' or event.keysym == 'Control_R':
            self.ctrl_pressed = False
            print("Клавиша Ctrl отпущена")

root = tk.Tk()
app = MyApp(root)
root.mainloop()